In [32]:
import pandas as pd
import glob
import os
import numpy as np


In [2]:
file_list = glob.glob(os.path.join('..', 'result', '**'))

In [117]:
df_result = pd.DataFrame()
for file_path in file_list : 
    split_path = file_path.replace('.xlsx', '').split('_')

    ym = int(split_path[1] + split_path[2])
    if 202208 > ym : 
        df = pd.read_excel(file_path)
        df = df[1:]
        now_columns = df.columns
        change_columns = [i for i in now_columns if '기준년월' not in i]
        df = df[change_columns]
        new_columns = ['bank', 'loan_type', 'grade_1_2', 'grade_3_4', 'grade_5_6', 'grade_7_8', 'grade_9', 'avg_inst', 'etc']
        df.columns = new_columns
        df['file_name'] = [file_path for i in range(len(df))]
        df['year'] = [split_path[1] for i in range(len(df))]
        df['month'] = [split_path[2] for i in range(len(df))]
    df_result = pd.concat([df_result, df])

In [118]:
for file_path in file_list : 
    split_path = file_path.replace('.xlsx', '').split('_')

    ym = int(split_path[1] + split_path[2])
    if ym > 202208 : 
        df = pd.read_excel(file_path)
        
        df.columns = ['bank', 'loan_type', 'grade_1', 'grade_2', 'grade_3', 'grade_4',
       'grade_5', 'grade_6', 'grade_7', 'grade_8', 'grade_9',
       'avg_inst', '평균신용점수', 'CB회사명', 'etc']
        df = df[1:]
        
        for i in range(1, 9, 2):
            col1 = f'grade_{i}'
            col2 = f'grade_{i+1}'
            new_col = f'grade_{i}_{i+1}'
            
            df[col1] = df[col1].replace('-', 0).astype(float)
            df[col2] = df[col2].replace('-', 0).astype(float)
            df[new_col] = df[[col1, col2]].apply(
                lambda x: np.mean(x[x != 0]) if any(x != 0) else 0,
                axis=1
            )
        df = df[['bank', 'loan_type', 'grade_1_2', 'grade_3_4', 'grade_5_6', 'grade_7_8',
                'grade_9', 'avg_inst', 'etc']]
        df['file_name'] = [file_path for i in range(len(df))]
        df['year'] = [split_path[1] for i in range(len(df))]
        df['month'] = [split_path[2] for i in range(len(df))]
        

    df_result = pd.concat([df_result, df])



In [119]:
df_result['bank'] = df_result['bank'].ffill().reset_index(drop = True)
df_result['grade_1_2'] = df_result['grade_1_2'].replace('-', 0).astype(float)
df_result['grade_3_4'] = df_result['grade_3_4'].replace('-', 0).astype(float)
df_result['grade_5_6'] = df_result['grade_5_6'].replace('-', 0).astype(float)
df_result['grade_7_8'] = df_result['grade_7_8'].replace('-', 0).astype(float)
df_result['grade_9'] = df_result['grade_9'].replace('-', 0).astype(float)
df_result['year'] = df_result['year'].astype(int)
df_result['month'] = df_result['month'].astype(int)

In [120]:
df_result.to_excel('./test.xlsx')

In [116]:
df_result

,bank,loan_type,grade_1_2,grade_3_4,grade_5_6,grade_7_8,grade_9,avg_inst,etc,file_name,year,month
1,BNK경남은행,대출금리,3.41,3.57,3.57,3.44,3.59,3.47,NaN,..\result\loan_2015_01.xlsx,2015,1
2,BNK경남은행,기준금리,2.19,2.16,2.22,2.16,2.15,2.18,NaN,..\result\loan_2015_01.xlsx,2015,1
3,BNK부산은행,가산금리,1.22,1.41,1.35,1.28,1.44,1.29,NaN,..\result\loan_2015_01.xlsx,2015,1
4,BNK부산은행,대출금리,3.78,3.76,3.78,3.79,3.79,3.78,NaN,..\result\loan_2015_01.xlsx,2015,1
5,BNK부산은행,기준금리,2.47,2.51,2.41,2.45,2.41,2.45,NaN,..\result\loan_2015_01.xlsx,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...
68,Sh수협은행,가감조정금리,0.00,0.00,0.00,0.00,0.00,-,NaN,..\result\loan_2025_07.xlsx,2025,7
69,iM뱅크(구 대구은행),대출금리,0.00,0.00,0.00,0.00,0.00,-,NaN,..\result\loan_2025_07.xlsx,2025,7
70,iM뱅크(구 대구은행),기준금리,0.00,0.00,0.00,0.00,0.00,-,NaN,..\result\loan_2025_07.xlsx,2025,7
71,iM뱅크(구 대구은행),가산금리,0.00,0.00,0.00,0.00,0.00,-,NaN,..\result\loan_2025_07.xlsx,2025,7
